# 1　查詢語法

- SELECT FROM

In [ ]:
SELECT * FROM customers;

SELECT customers.first_name, customer.last_name FROM customers;

`SELECT *` 代表選擇全部

- AS

In [ ]:
SELECT customers.first_name AS "First name" FROM customers;

使用 `AS` 宣告別名 (Alias) ，改變欄位或表單的顯示名稱

此名稱為單純字串，使用雙引號 `"`

- WHERE

In [ ]:
SELECT * FROM customers WHERE customers.name='Rolf';

過濾結果，對象鍵值使用單引號 `'` (某些資料庫可接受雙引號但 PostgreSQL 不行)

可以再加入 `OR` 或 `AND` 進一步限縮

- LIMIT

In [ ]:
SELECT * FROM customers LIMIT 5;

限制結果的數量只顯示前 5 個

- UPDATE

In [ ]:
UPDATE items SET price=50 WHERE id=3;

將 `items` 表單中 `id` 為 3 的商品之價格更新為 50

注意：若不用 `WHERE` 限縮範圍，表單中的所有商品價格都會被設成 50

- DELETE FROM

In [ ]:
DELETE FROM items WHERE id=2;

PostgreSQL 會檢查項目是否有被其他表單參考，若被參考則無法刪除，必須先刪除參考源

- LIKE

In [ ]:
SELECT * FROM customers WHERE last_name LIKE '%o__';

利用 `LIKE` 及外卡字元進行鍵值過濾：`%` 代表一個或以上的字元， `_` 代表一個字元

故 `%o__` 可以包含 Starlord、moon，但不包含 Python

- INDEX

一般而言使用鍵值進行搜尋，但在龐大資料庫系統中會花較多時間

可以預先建立好索引值 (index) 來加速未來的搜尋 (通常是基於二元樹演算法)

當然這會花掉額外資源，新增資料時也會變得較耗時

In [ ]:
CREATE INDEX user_name_index ON customers(name);

`user_name_index` 可以任意命名

如此 PostgreSQL 會依據表單 customers 中的 name 欄位進行索引值建立

必要時亦可同時針對多個欄位建立索引值，以利未來的多欄位同時搜尋


In [ ]:
REINDEX INDEX user_name_index;

重新建立索引值，來清除遇到的索引值錯誤

***

# 2-1　表單語法 - JOIN

- INNER JOIN ON

In [ ]:
SELECT * FROM customers INNER JOIN orders ON customers.ID=Orders.customer_ID

根據指定鍵值取兩個表單的__交集__，形成新的表單

新的表單會有 `customers.ID`、`orders.ID`、`orders.customer_ID` 三個編號欄位

三個欄位都有對應值的項目才會被列出

- LEFT JOIN ON

In [ ]:
SELECT * FROM customers LEFT JOIN orders ON customers.ID=Orders.customer_ID

以__左邊表單__(`customers`)為主，依照 `customers.ID` 列出，並優先列出交集

若無對應 `orders.ID`、`orders.customer_ID` 則該處內容留白

- RIGHT JOIN ON


In [ ]:
SELECT * FROM customers RIGHT JOIN orders ON customers.ID=Orders.customer_ID

以__右邊表單__(`orders`)為主，依照 `orders.ID` 列出，並優先列出交集

若無對應 `customers.ID`、`orders.customer_ID` 則該處內容留白

- FULL JOIN ON

In [ ]:
SELECT * FROM customers FULL JOIN orders ON customers.ID=Orders.customer_ID

根據指定鍵值取兩個表單的__聯集__，形成新的表單

- 多重加入

In [ ]:
SELECT * FROM items 
INNER JOIN purchases ON items.id = purchases.item_id
INNER JOIN customers ON purchases.customer_id = customers.id;

- GROUP BY

In [ ]:
SELECT customers.first_name, COUNT(purchases.id) FROM customers 
LEFT JOIN purchases ON customers.id = purchases.customer_id
GROUP BY customers.id;

把 `customer.id` 相同的項目合併起來

再用 `COUNT` 函式 來計算有幾項 相同 `customer.id` 、不同 `purchases.id` 的項目
被合併了 

被合併的項目，若原先有不同資料則會消失，但可以存取這些資料的統計值如  
`COUNT()`、`SUM()`、 `AVG()`、`MAX()`


- ORDER BY

In [ ]:
SELECT items.name, items.price FROM items ORDER BY items.price DESC;

`DESC` 降序排列、`ASC` 升序排列

- HAVING

In [ ]:
SELECT customers.first_name, COUNT(purchases.id)
FROM customers 
INNER JOIN purchases ON customers.id = purchases.customer_id
GROUP BY customers.id
HAVING COUNT(purchases.id) > 3;

如果我們需要「項目數量超過 3」之類的條件

因為統計值並不直接是表單內的資料，並不能使用 `WHERE` 來查詢，必須使用 `HAVING`

# 2-2　表單語法 - 新增刪除

- CREATE

In [ ]:
CREATE TABLE public.users (
id integer PRIMARY KEY,
name character varying(255) NOT NULL
)

建立名為 `users` 的表單， `public.` 是預設 schema，可以省略

很大量的數字可以使用 `bigint`

`character varing(255) NOT NULL` 表示為最長 255 字元的資料，且不得為空

若不想限制長度就使用 `text`

也可以不直接在 id 後面指定主鍵，改為這樣寫：

In [ ]:
CREATE TABLE public.users (
id integer,
name character varying(255) NOT NULL,
CONSTRAINT user_id_pkey PRIMARY KEY(id)
)

如此做的原因是可以指定很多個主鍵，`user_id_pkey` 可以是任意的命名

In [ ]:
CREATE TABLE public.videos (
id integer PRIMARY KEY,
user_id integer REFERENCES public.users,
name character varying(255) NOT NULL
)

`REFERENCES` 指定外鍵及其參考的表單

- INSERT INTO

In [ ]:
INSERT INTO public.users (id, name) VALUES (2, 'Mary');

若輸入的資料完全對應表單各欄位，則 `(id, name)` 可以省略

但若只對應某些欄位，則需要指定資料是填入哪些欄位

In [ ]:
INSERT INTO public.videos VALUES (2, 999, 'New video');

若沒有第 999 位使用者，因外鍵無法參考，則無法插入

- DROP

In [ ]:
DROP TABLE public.users RESTRICT;

刪除表單，`RESTRICT` 為預設，可省略，會檢查表單間的關係

由於此時表單 `videos` 參考到 `users`，故無法進行刪除

In [ ]:
DROP TABLE public.users CASCADE;

使用 `CASCADE` 即可刪除表單 `users`

如此會刪除表單 `videos` 的參考關係，但不會刪除其資料

- IF EXISTS

In [ ]:
DROP TABLE IF EXISTS public.users;

不確定是否存在時，可使用 `IF EXISTS` 條件式

# 2-3　表單語法 - 序列

- SEQUENCE (PostgreSQL Only)

In [ ]:
CREATE SEQUENCE users_id_seq START 3;

如此會產生從 3 開始的序列，若不指定則預設從 1 開始

- ALTER

In [ ]:
ALTER TABLE public.users
ALTER COLUMN id
SET DEFAULT nextval('users_id_seq');

將表單 `users` 的 `id` 欄位預設值改為 `users_id_seq` 的下一個值

- OWNED BY

In [ ]:
ALTER SEQUENCE 'users_id_seq' OWNED BY public.users.id;

當刪除表單 `users` 時，其所擁有的序列 `users_id_seq` 也會一併被刪除

- SERIAL

In [ ]:
CREATE TABLE test (
id SERIAL PRIMARY KEY,
name text
);

INSERT INTO test(name) VALUES ('John');

`SERIAL` 背後亦是基於 `SEQUENCE` 建立的，使用上較為方便

***

# 3　VIEW

In [ ]:
CREATE VIEW total_revenue_per_customer AS

SELECT customers.first_name, SUM(items.price) FROM customers 
INNER JOIN purchases ON customers.id = purchases.customer_id
INNER JOIN items ON purchases.item_id = items.id
GROUP BY customers.id;

以 `VIEW` 建立一個特定的結果表單，可以免去每次輸入一長串的指令

之後再如一般表單的查詢方式：

In [ ]:
SELECT * FROM total_revenue_per_customer

若 `items.price` 更新，重新查詢一次 `total_revenue_per_customer` 結果也會跟著更新

亦即 `VIEW` 是 updatable，可以想像成是一個自定義查詢函數

In [ ]:
CREATE VIEW expensive_item AS
SELECT * FROM items WHERE price > 100;

INSERT INTO expensive_items (id, name, price)
VALUES (9, 'Pencil', 15);

可以如一般表單插入資料，但要注意：

- GROUP BY 處理過的表單可能無法插入已融合的欄位項目

- 上述 Pencil 縱使價格小於 100，仍會成功加入 items 表單當中，只是在 VIEW 中因過濾看不到 

In [ ]:
INSERT INTO expensive_items (id, name, price)
VALUES (9, 'Pencil', 15)
WITH LOCAL CHECK OPTION;

若加入 `WITH LOCAL CHECK OPTION` (PostgreSQL Only)

則系統會不允許價格小於 100 的鉛筆資料插入

In [ ]:
CREATE VIEW non_luxury_items AS
SELECT * FROM expensive_items WHERE price < 1000
WITH LOCAL CHECK OPTION;

可以從 VIEW 再建立另一個 VIEW

In [ ]:
INSERT INTO non_luxury_items (id, name, price)
VALUES (9, 'Pencil', 15)
WITH LOCAL CHECK OPTION;

縱使 `non_luxury_items` 是基於 `expensive_items` 建立的

但因為是 `LOCAL CHECK OPTION`，鉛筆的價格 15 滿足 `non_luxury_items` 小於 1000 的條件

故鉛筆可成功插入 `items` 表單

如果也想檢查 `expensive_items` 的條件，則需要改成 `WITH CASCADED CHECK OPTION` 

In [ ]:
DROP VIEW expensive_items;

刪除 VIEW 的方法與一般表單相同

***

# 4　時間與 Enum

- 時間

In [ ]:
SELECT NOW();

會回傳 ISO 8601 格式 `YYYY-MM-DD HH-MI-SS` 的 時間戳記(timestamp)

如 `2018-06-13 14:12:25.993795+08`

代表現在台灣時間為 2018-06-13 下午 2 點 12 分 25 秒， `+08` 是指 UTC+8 時區

In [ ]:
SELECT TO_CHAR(NOW(), 'Day DDth Month, DD-MM-YYYY');

可以將時間戳記更改成其他格式，回傳成為如 `Sunday  3rd June    , 03-06-2018` 的格式

若寫成 `FMDay`、 `FMMonth` 可以移除對齊用的空白


In [ ]:
SELECT TO_TIMESTAMP('Sunday 3rd June, 2018 15:00:00',
                    'FMDay DDth FMonth, YYYY HH:MI:SS')

可以將其他格式的時間資料轉回標準格式

- Enum

In [ ]:
CREATE TYPE mood AS ENUM('happy', 'ok', 'sad');

藉由 `ENUM` 來限制允許的資料內容

In [ ]:
CREATE TABLE students (
name character varing(255),
current_mood mood
);

如此表單 `students` 當中會有一欄 `current_mood` 其資料格式為 `mood`

且只允許輸入 `happy` 、 `ok` 、 `sad`

要注意的是 ENUM 是有順序性的，上例中可以看成 happy = 0 、 ok = 1 、 sad = 2 

In [ ]:
SELECT * FROM students WHERE current_mood < 'ok';

會篩選出 `current_mood` 為 `happy` 的學生

***

# 5　整合應用 - 巢狀選擇

考慮下面這個看起來很直覺的查詢：

In [ ]:
SELECT * FROM items WHERE price > AVG(price);

但可惜的是這不是正確的語法

而且 `HAVING` 必須搭配 `GROUP BY` 才能使用，也不是正確寫法

這時必須使用巢狀選擇：

In [ ]:
SELECT * FROM items WHERE price > ( SELECT AVG(items.price) FROM items );

另一個巢狀選擇例子：

In [ ]:
CREATE VIEW expensive_items_diff AS
SELECT items.name, 
items.price - ( SELECT AVG(items.price) FROM items WHERE price > 100)
FROM items WHERE price > 100;

SELECT * FROM expensive_items_diff

可以得到 價格超過 100 的高價商品 其價格 與 高價商品平均價格 的差距